# Used for update in 2022/5/2

## Function definition

In [9]:
import tweepy
import pickle
import time
import couchdb3

In [10]:
### definitions
def reform_tweet(raw_tweet):
    """
    Reform the tweet data into a dictionary form.
    Each tweet is a dictionary contains: tweet_id, text, author_id, create_at, geo
    """
    tweet_dict = dict()
    tweet_dict['_id'] = str(raw_tweet.id)
    tweet_dict['author_id'] = raw_tweet.author_id
    tweet_dict['text'] = raw_tweet.text
    tweet_dict['created_at'] = str(raw_tweet.created_at)
    tweet_dict['geo'] = raw_tweet.geo

    return tweet_dict


def preprocess(raw_tweet, tweet_list):
    """
    Process all tweets in one turn, remove tweets withou geo info
    """
    for data in raw_tweet:
        tweet_list.append(reform_tweet(data))
    
    return tweet_list


def get_tweet_1(query, token, tweets_each_turn, next_page = None):
    """
    Get tweet using Twitter API 2.0. 
    Using keywords(query) as a main parameter
    Caputure tweet_id, text, geo, author_id, create_at
    """
    client = tweepy.Client(bearer_token= token)
    # each time capture 100 tweets
    tweets = client.search_recent_tweets(query=query, tweet_fields=['author_id', 'created_at', 'geo'], max_results=tweets_each_turn, next_token=next_page)

    print(tweets.meta)

    return tweets.data, tweets.meta['next_token']


def crawler(query, tokens, tweets_each_turn, turns, client, db_name, db_name2):
    """
    Main function. Use query as filter, token for authocation, and number of tweets each turn
    Turns suggest how many turns this function will run
    Get tweets from Twitter
    Refrom tweets to dictionary type
    Save tweets to CouchDB
    Automatically repeat this process.
    For one token, speed limit is 900 tweets/15min(1/sec)
    """

    time_gap = int (tweets_each_turn / len(tokens)) + 1
    
    count = 0
    next_page = None
    tweet_list = []
    while(count < turns):
        for token in tokens:
            data, next_page = get_tweet_1(query, token, tweets_each_turn, next_page)
            tweet_list = preprocess(data, tweet_list)
            time.sleep(time_gap)
        
        count = count + 1

    #save_to_couchDB(client, tweet_list, db_name, db_name2)

    output = open('tweetdata.pkl','wb')
    pickle.dump(tweet_list, output)
    output.close()
    
    return next_page



def save_to_couchDB(client, tweet_data, db_name, db_name2):
    """
    Save tweets to CouchDB, remove all duplicates
    Save tweets without geo info to db_name
    Save tweets with geo info to db_name2
    """
    if(client.up() == True):
        print("Connected to CouchDB")
    else:
        print("Unable to connect to CouchDB")
        return

    if( db_name not in client.all_dbs()):
        print("No database:" + db_name + ", create one first")
        client.create(db_name)

    if( db_name2 not in client.all_dbs()):
        print("No database:" + db_name2 + ", create one first")
        client.create(db_name2)
    
    db1 = client.get(db_name)
    count1 = 0

    db2 = client.get(db_name2)
    count2 = 0

    for data in tweet_data:
        if(data['geo'] == None):
            if(data['_id'] not in db1):
                db1.save(data)
                count1 += 1
        
        else:  #store tweets with geo to another database
            if(data['_id'] not in db2):
                db2.save(data)
                count2 += 1
    
    print(str(count1) + " tweets(no geo) are successfully saved to database" + db_name)
    print(str(count2) + " tweets(with geo) are successfully saved to database" + db_name2)
    return




def crawler_2(query, tokens, tweets_each_turn, turns, client, db_name):
    """
    Main function. Use query as filter, token for authocation, and number of tweets each turn
    Turns suggest how many turns this function will run
    Get tweets from Twitter
    Refrom tweets to dictionary type
    Save tweets to CouchDB
    Automatically repeat this process.
    For one token, speed limit is 900 tweets/15min(1/sec)
    """

    time_gap = int (tweets_each_turn / len(tokens)) + 1
    
    count = 0
    next_page = None
    tweet_list = []
    while(count < turns):
        for token in tokens:
            data, next_page = get_tweet_1(query, token, tweets_each_turn, next_page)
            tweet_list = preprocess(data, tweet_list)
            time.sleep(time_gap)
        
        count = count + 1

    #save_to_couchDB_2(client, tweet_list, db_name)
    output = open('topicB.pkl','wb')
    pickle.dump(tweet_list, output)
    output.close()
    
    return next_page



def save_to_couchDB_2(client, tweet_data, db_name):
    """
    Save tweets to CouchDB, remove all duplicates
    Save tweets with geo info to db_name
    Filter is based on coorinates
    Used for specific scenario
    """

    geo_filter = [144.33363404800002, -38.50298801599996, 145.8784120140001, -37.17509899299995]

    if(client.up() == True):
        print("Connected to CouchDB")
    else:
        print("Unable to connect to CouchDB")
        return

    if( db_name not in client.all_dbs()):
        print("No database:" + db_name + ", create one first")
        client.create(db_name)
    
    db1 = client.get(db_name)
    count1 = 0

    for data in tweet_data:
        if(data['geo'] != None):#only store tweets with geo to database
            if('coordinates' in data['geo'].keys()):
                if(data['_id'] not in db1):
                    latitude = data['geo']['coordinates']['coordinates'][0] 
                    longitude = data['geo']['coordinates']['coordinates'][1] 
                    if( latitude >= geo_filter[0] and latitude <= geo_filter[2]):
                        if(longitude >= geo_filter[1] and longitude <= geo_filter[3]):
                            db1.save(data)
                            count1 += 1

    print(str(count1) + " tweets(with geo) are successfully saved to database" + db_name)
    return

## Test setup

In [11]:
#### Set up
BEARER_TOKEN = ["AAAAAAAAAAAAAAAAAAAAALWBbQEAAAAA%2FbQ0tpIE3uy14yUmYU0AiocoH6c%3DDkX3Fl2TdMFgRBCivYCSMajfqglkm8DkyylcAXkUFFceAIOBRB"]


query1 = '(Melbourne OR melbourne) lang:en'
query2 = '#Melbourne lang:en'
query3 = 'suiside OR depression OR anxiety OR dying OR hallucination lang:en'

client = couchdb3.Server(
    "http://172.26.132.196:5984",
    user="admin",
    password="admin"
)
db_name = "raw_tweets"
db_name2 = "geo_tweets"



## Test

In [10]:
if(client.up()):
    while(True): ## infinite loop
        #next_tokens = crawler(query1, BEARER_TOKEN, 50, 5, client, db_name, db_name2)
        next_tokens = crawler(query2, BEARER_TOKEN, 50, 6, client, db_name, db_name2)
        break

{'newest_id': '1521049510113583105', 'oldest_id': '1521040652699586565', 'result_count': 50, 'next_token': 'b26v89c19zqg8o3fpywl7jy4a4ognw9h4zp81htv3wvb1'}
{'newest_id': '1521040419273945088', 'oldest_id': '1521021839807553538', 'result_count': 50, 'next_token': 'b26v89c19zqg8o3fpywl7jxiqsz6texxkqb5b1iqf2gsd'}
{'newest_id': '1521021839639781376', 'oldest_id': '1521007292908707840', 'result_count': 50, 'next_token': 'b26v89c19zqg8o3fpywl7jwxf0orijf3rtklr7564m5bx'}
{'newest_id': '1521006881203900416', 'oldest_id': '1520989610897461254', 'result_count': 50, 'next_token': 'b26v89c19zqg8o3fpywksxjx8ffe8vurhawxekmewowvx'}
{'newest_id': '1520989465078296577', 'oldest_id': '1520965848898416640', 'result_count': 50, 'next_token': 'b26v89c19zqg8o3fpywksxjbhim7mno8q84bfetcphif1'}
{'newest_id': '1520965808649740289', 'oldest_id': '1520948662267830272', 'result_count': 50, 'next_token': 'b26v89c19zqg8o3fpywksxiq17b5m13dw2p4xp6chsd4t'}


In [12]:
input = open('tweetdata.pkl','rb')
data1 = pickle.load(input)
input.close()

In [13]:
for data in data1:
    if(data['geo'] != None):
        if('coordinates' in data['geo'].keys()):
            print(data['geo'])

{'place_id': '01864a8a64df9dc4', 'coordinates': {'type': 'Point', 'coordinates': [144.9630576, -37.8136276]}}


In [13]:
save_to_couchDB_2(client, data1, "test")

Connected to CouchDB
No database:test, create one first
1 tweets(with geo) are successfully saved to databasetest


In [5]:
if(client.up()):
    while(True): ## infinite loop
        #next_tokens = crawler(query1, BEARER_TOKEN, 50, 5, client, db_name, db_name2)
        next_tokens = crawler_2(query3, BEARER_TOKEN, 50, 6, client, db_name)
        break

{'newest_id': '1521057747806130181', 'oldest_id': '1521057650519293952', 'result_count': 50, 'next_token': 'b26v89c19zqg8o3fpywl7jyfd6euaxm5eys92oft6cigt'}
{'newest_id': '1521057649055657985', 'oldest_id': '1521057545938481154', 'result_count': 50, 'next_token': 'b26v89c19zqg8o3fpywl7jyfd674puiuk0ywyy55gs9rx'}
{'newest_id': '1521057540397797376', 'oldest_id': '1521057458982211584', 'result_count': 50, 'next_token': 'b26v89c19zqg8o3fpywl7jyfd5zha88qhh6e2tfc4yut9'}
{'newest_id': '1521057458843697152', 'oldest_id': '1521057386571743233', 'result_count': 50, 'next_token': 'b26v89c19zqg8o3fpywl7jyfd5rvzmd704p9p1ks35g1p'}
{'newest_id': '1521057386055749637', 'oldest_id': '1521057319773253632', 'result_count': 50, 'next_token': 'b26v89c19zqg8o3fpywl7jyfd5rofveduw9uf2ogmxv99'}
{'newest_id': '1521057316099051521', 'oldest_id': '1521057225057480705', 'result_count': 50, 'next_token': 'b26v89c19zqg8o3fpywl7jyfd5k0z6otpjapxfp8e1cvx'}


In [6]:
input = open('topicB.pkl','rb')
data2 = pickle.load(input)
input.close()

In [8]:
for data in data2:
    if(data['geo'] != None):
        if('coordinates' in data['geo'].keys()):
            print(data['geo'])